In [ ]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df = spacex_df.rename(columns= {'PAYLOAD_MASS__KG_':'Payload Mass (kg)'})
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options = [
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                            ],
                                            value = 'All',
                                            placeholder = 'All Sites',
                                            searchable = True,
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min = min_payload, max = max_payload, step = 100,
                                                marks = {0: '0', max_payload: '10000'},
                                                value = [1000, 10000],
                                                dots = True,
                                                vertical = False,
                                                tooltip = {'always visibel': False, 'placement': 'bottom'}),

                                # # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id = 'success-pie-chart', component_property = 'figure'),
                Input(component_id = 'site-dropdown', component_property = 'value'))
def get_pie_chart(site):
    if site == 'ALL':
        data = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
        fig = px.pie(data, values = 'class', names = 'Launch Site', title = 'Total Success Launches By Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == site]
        data = filtered_df.groupby('class').count().reset_index()
        fig = px.pie(data, values = 'Launch Site', names = 'class', title = 'Total Success Launches for site' + ' {}'.format(site))
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
    [Input(component_id = 'site-dropdown', component_property = 'value'), Input(component_id = 'payload-slider', component_property = 'value')])

def get_scatter_chart(site, payload):
    data = spacex_df.loc[payload[1]:payload[0]]
    if site == 'ALL':
        fig = px.scatter(data, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category', title = 'Correlation between Payload and Success for all Sites')
        return fig
    else:
        data = spacex_df[spacex_df['Launch Site'] == site]
        fig = px.scatter(data, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category', title = 'Correlation between Payload and Success for' + ' {}'.format(site))
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug = True)
